In [1]:
import requests
import time
import json
from tools import get_key

GENIUS_API_TOKEN = get_key("Client_access_genius")

headers = {
    'Authorization': f'Bearer {GENIUS_API_TOKEN}'
}

In [2]:
# Test de récupération de la liste de sons d'un artiste via l'API Genius
def get_artist_songs(artist_id, page=1):
    url = f'https://api.genius.com/artists/{artist_id}/songs'
    params = {
        'page': page,
        'sort': 'popularity'
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        return None

In [22]:
def get_artist_featurings(artist_id, max_pages=1):
    songs = []
    for page in range(1, max_pages + 1):
        url = f'https://api.genius.com/artists/{artist_id}/songs'
        params = {
            'page': page,
            'sort': 'popularity'
        }
        response = requests.get(url, headers=headers, params=params)
        if response.status_code != 200:
            print(f"Erreur page {page} : {response.status_code}")
            break

        data = response.json()
        song_list = data.get('response', {}).get('songs', [])

        for song in song_list:
            title = song.get('title')
            primary_artist = song.get('primary_artist', {}).get('name')
            featured_artists = song.get('featured_artists', [])

            # Extraire les noms des artistes (principal + feats)
            authors = [primary_artist] + [artist['name'] for artist in featured_artists]
            if len(authors) > 1:
                songs.append((title, authors))

        if not song_list:
            break

    return songs




In [23]:
songs = get_artist_featurings(13325, 100)
songs

[('Sapés comme jamais (Pilule Bleue)', ['GIMS', 'Niska']),
 ('Loup garou', ['GIMS', 'Sofiane']),
 ('\u200b\u200b\u200b\u200bavf', ['Stromae', 'GIMS', 'OrelSan']),
 ('Game Over', ['Vitaa', 'GIMS']),
 ('Mi Gna', ['GIMS', 'Super Sako', 'Spitakci Hayko']),
 ('Je te pardonne (Pilule Bleue) [Sia Remix]', ['GIMS', 'Sia']),
 ('Christophe', ['OrelSan', 'GIMS']),
 ('Ça marche', ['GIMS', 'The Shin Sekaï']),
 ('Longueur d’avance', ['Booba', 'GIMS']),
 ('Sharingan', ['GIMS', 'The Shin Sekaï', 'Insolent', 'OrelSan']),
 ('La même', ['GIMS', 'Vianney']),
 ('Par amour', ['Dadju', 'GIMS']),
 ('One Shot', ['GIMS', 'Dry']),
 ('Corazón', ['GIMS', 'Lil Wayne', 'French Montana']),
 ('Hola Señorita', ['GIMS', 'Maluma']),
 ('Melrose Place', ['Sexion d’Assaut', 'L.I.O. Petrodollars']),
 ('Wati by Night', ['Sexion d’Assaut', 'Dry']),
 ('Outsider', ['GIMS', 'Djuna Family']),
 ('Loin (Pilule Violette)', ['GIMS', 'SoFLY', 'Nius', 'Dany Synthé']),
 ('Ma fierté', ['Dadju', 'GIMS', 'Alonzo']),
 ('Longue vie (Pilule Ro